In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install rtree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pygeos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import lightgbm as lgb
import geopandas as gpd
import rtree
import pygeos
import datetime

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-4-33908bcd5e6a>:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  impo

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
traffic_bordeaux = pd.read_csv('/content/drive/MyDrive/utd_19_bordeaux.csv')
traffic_constance = pd.read_csv('/content/drive/MyDrive/utd_19_constance.csv')
detectors = pd.read_csv('/content/drive/MyDrive/detectors_public.csv')

In [7]:
min_list = []
for i in range(96):
  min_list.append(i * 900)

In [8]:
traffic_bordeaux = traffic_bordeaux[['day', 'interval', 'flow', 'detid']]
traffic_constance = traffic_constance[['day', 'interval', 'flow', 'detid']]

In [9]:
traffic_bordeaux = traffic_bordeaux[traffic_bordeaux['interval'].isin(min_list)]
traffic_constance = traffic_constance[traffic_constance['interval'].isin(min_list)]

In [10]:
traffic_bordeaux['tdelta'] = traffic_bordeaux['interval'].round().apply(pd.to_timedelta, errors='coerce', unit='s')
traffic_bordeaux['time'] = (pd.Timestamp('now').normalize() + traffic_bordeaux['tdelta']).dt.time
traffic_bordeaux['time'] = traffic_bordeaux['time'].astype(str)
traffic_bordeaux['datetime'] = pd.to_datetime(traffic_bordeaux['day'] + ' ' + traffic_bordeaux['time'])
traffic_bordeaux = traffic_bordeaux[['datetime', 'flow', 'detid']]

traffic_constance['tdelta'] = traffic_constance['interval'].round().apply(pd.to_timedelta, errors='coerce', unit='s')
traffic_constance['time'] = (pd.Timestamp('now').normalize() + traffic_constance['tdelta']).dt.time
traffic_constance['time'] = traffic_constance['time'].astype(str)
traffic_constance['datetime'] = pd.to_datetime(traffic_constance['day'] + ' ' + traffic_constance['time'])
traffic_constance = traffic_constance[['datetime', 'flow', 'detid']]

In [11]:
detectors = detectors[(detectors['citycode'] == 'constance') | (detectors['citycode'] == 'bordeaux')]

In [12]:
detectors_bordeaux = detectors[(detectors['citycode'] == 'bordeaux')]
detectors_constance = detectors[(detectors['citycode'] == 'constance')]

In [13]:
meteo_bordeaux = pd.read_csv('/content/drive/MyDrive/meteo_bordeaux.csv')
meteo_constance = pd.read_csv('/content/drive/MyDrive/meteo_constance.csv')

In [14]:
geo_bordeaux = pd.read_csv('/content/drive/MyDrive/geo_bordeaux.csv')
geo_constance = pd.read_csv('/content/drive/MyDrive/geo_constance.csv')

In [15]:
traffic_bordeaux['datetime'] = traffic_bordeaux['datetime'].astype('datetime64[ns]')
meteo_bordeaux['time'] = meteo_bordeaux['time'].astype('datetime64[ns]')
bord_traf_meteo = pd.merge(traffic_bordeaux, meteo_bordeaux, left_on='datetime', \
                          right_on='time', how='inner')
bord_traf_geo_meteo = pd.merge(bord_traf_meteo, geo_bordeaux, on = 'detid')
counts = bord_traf_geo_meteo['detid'].value_counts()
counts = counts[counts > 668]




In [16]:
df_bordeaux = pd.DataFrame()
for detid in bord_traf_geo_meteo['detid'].unique():
  det_subset = bord_traf_geo_meteo[bord_traf_geo_meteo["detid"] == detid]
  det_subset = det_subset.sort_values(by='datetime')
  int_list = det_subset['flow'].tolist()
  int_mem = det_subset['flow'].tolist()
  del int_list[:96] 
  del int_mem[-96:]
  det_subset = det_subset.iloc[96:]
  det_subset['int_mem'] = int_mem
  det_subset['int'] = int_list
  det_subset = det_subset.drop(columns = ['flow', 'datetime', 'time', 'detid'])
  df_bordeaux = pd.concat([df_bordeaux, det_subset])

In [17]:
df_bordeaux.to_csv('/content/drive/MyDrive/df_bordeaux.csv')


In [18]:
traffic_constance['datetime'] = traffic_constance['datetime'].astype('datetime64[ns]')
meteo_constance['time'] = meteo_constance['time'].astype('datetime64[ns]')
const_traf_meteo = pd.merge(traffic_constance, meteo_constance, left_on='datetime', \
                          right_on='time', how='inner')
const_traf_geo_meteo = pd.merge(const_traf_meteo, geo_constance, on = 'detid')
counts = const_traf_geo_meteo['detid'].value_counts()
#print(counts)
counts = counts[counts > 668]

In [19]:
df_constance = pd.DataFrame()
for detid in const_traf_geo_meteo['detid'].unique():
  det_subset = const_traf_geo_meteo[const_traf_geo_meteo["detid"] == detid]
  det_subset = det_subset.sort_values(by='datetime')
  int_list = det_subset['flow'].tolist()
  int_mem = det_subset['flow'].tolist()
  del int_list[:96] 
  del int_mem[-96:]
  det_subset = det_subset.iloc[96:]
  det_subset['int_mem'] = int_mem
  det_subset['int'] = int_list
  det_subset = det_subset.drop(columns = ['flow', 'datetime', 'time', 'detid'])
  df_constance = pd.concat([df_constance, det_subset])

In [20]:
print(df_constance)

            u10       v10        t2m        ssr            tp  roads_1_25  \
96    -1.155001 -1.243369  269.89578  5027890.5  3.493857e-06         0.0   
97    -1.130275 -1.198529  269.84240  3770918.0  2.620392e-06         0.0   
98    -1.105549 -1.153689  269.78906  2513945.2  1.746928e-06         0.0   
99    -1.080823 -1.108849  269.73570  1256972.6  8.734642e-07         0.0   
100   -1.056098 -1.064009  269.68230        0.0  0.000000e+00         0.0   
...         ...       ...        ...        ...           ...         ...   
81613  1.450095  0.581100  273.98032  5449942.5  3.493857e-06         0.0   
81614  1.460346  0.588313  273.81195  5449942.5  3.493857e-06         0.0   
81615  1.470596  0.595526  273.64355  5449942.5  3.493857e-06         0.0   
81616  1.480846  0.602738  273.47520  5449942.5  3.493857e-06         0.0   
81617  1.491097  0.609951  273.30682  5449942.5  3.493857e-06         0.0   

       residential_a_25  roads_3_25  residential_a_50  roads_3_100  \
96   

In [21]:
df_constance.to_csv('/content/drive/MyDrive/df_constance.csv')
